## RNN

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from tqdm.notebook import tqdm

In [2]:
n_hidden = 35
lr = 0.01
epochs = 1000

string = "hello pytorch. how long can a rnn cell remember?"
chars= "abcdefghijklmnopqrstuvwxyz ?!.,:;01"
char_list = [i for i in chars]
n_letters = len(char_list)

In [7]:
def string_to_onehot(string):
    start = np.zeros(shape=len(char_list), dtype=int)
    end = np.zeros(shape=len(char_list), dtype=int)
    start[-2] = 1
    end[-1] = 1
    for i in string:
        idx = char_list.index(i)
        zero = np.zeros(shape=n_letters, dtype=int)
        zero[idx]=1
        start = np.vstack([start, zero])
    output = np.vstack([start, end])
    return output

In [9]:
def onehot_to_word(onehot_1):
    onehot = torch.Tensor.numpy(onehot_1)
    return char_list[onehot.argmax()]

In [15]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.i2h = nn.Linear(input_size, hidden_size)
        self.h2h = nn.Linear(hidden_size, hidden_size)
        self.i2o = nn.Linear(hidden_size, output_size)
        self.act_f = nn.Tanh()
        
    
    def forward(self, input, hidden):
        hidden = self.act_f(self.i2h(input)+self.h2h(hidden))
        output = self.i2o(hidden)
        return output, hidden
    
    
    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)

In [16]:
rnn = RNN(n_letters, n_hidden, n_letters)
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)

In [23]:
one_hot = torch.from_numpy(string_to_onehot(string)).type_as(torch.FloatTensor())

for i in tqdm(range(epochs)):
    rnn.zero_grad()
    total_loss = 0
    hidden = rnn.init_hidden()
    
    for j in range(one_hot.size()[0]-1):
        input_ = one_hot[j:j+1, :]
        target = one_hot[j+1]
        
        output, hidden = rnn.forward(input_, hidden)
        loss = loss_func(output.view(-1), target.view(-1))
        total_loss += loss
        input_ = output
        
    total_loss.backward()
    optimizer.step()
    
    if i % 10 == 0:
        print(total_loss)

tensor(0.0026, grad_fn=<AddBackward0>)
tensor(0.0025, grad_fn=<AddBackward0>)
tensor(0.0024, grad_fn=<AddBackward0>)
tensor(0.0023, grad_fn=<AddBackward0>)
tensor(0.0023, grad_fn=<AddBackward0>)
tensor(0.0204, grad_fn=<AddBackward0>)
tensor(0.0049, grad_fn=<AddBackward0>)
tensor(0.0032, grad_fn=<AddBackward0>)
tensor(0.0027, grad_fn=<AddBackward0>)
tensor(0.0024, grad_fn=<AddBackward0>)
tensor(0.0022, grad_fn=<AddBackward0>)
tensor(0.0021, grad_fn=<AddBackward0>)
tensor(0.0020, grad_fn=<AddBackward0>)
tensor(0.0020, grad_fn=<AddBackward0>)
tensor(0.0085, grad_fn=<AddBackward0>)
tensor(0.0068, grad_fn=<AddBackward0>)
tensor(0.0034, grad_fn=<AddBackward0>)
tensor(0.0024, grad_fn=<AddBackward0>)
tensor(0.0021, grad_fn=<AddBackward0>)
tensor(0.0020, grad_fn=<AddBackward0>)
tensor(0.0019, grad_fn=<AddBackward0>)
tensor(0.0018, grad_fn=<AddBackward0>)
tensor(0.0018, grad_fn=<AddBackward0>)
tensor(0.0017, grad_fn=<AddBackward0>)
tensor(0.0017, grad_fn=<AddBackward0>)
tensor(0.0016, grad_fn=<A

In [24]:
start = torch.zeros(1, len(char_list))
start[:, -2] = 1

with torch.no_grad():
    hidden = rnn.init_hidden()
    input_ = start
    output_string = ""
    for i in range(len(string)):
        output, hidden = rnn.forward(input_, hidden)
        output_string += onehot_to_word(output.data)
        input_ = output

print(output_string)

hello pyr rerememememererereremememem rererememe


## LSTM

In [27]:
!rm -r data
import os 

try:
  os.mkdir("./data")
except:
  pass

!wget https://raw.githubusercontent.com/dmlc/web-data/master/mxnet/tinyshakespeare/input.txt -P ./data

rm: 'data'를 지울 수 없음: 그런 파일이나 디렉터리가 없습니다
--2021-01-13 09:28:45--  https://raw.githubusercontent.com/dmlc/web-data/master/mxnet/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.76.133
접속 raw.githubusercontent.com (raw.githubusercontent.com)|151.101.76.133|:443... 접속됨.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘./data/input.txt’

input.txt           100%[===================>]   1.06M  2.18MB/s    in 0.5s    

2021-01-13 09:28:47 (2.18 MB/s) - ‘./data/input.txt’ saved [1115394/1115394]



In [28]:
import torch
import torch.nn as nn

In [30]:
import unidecode
import string
import random
import re
import time, math
from tqdm.notebook import tqdm

In [31]:
num_epochs = 2000
print_every = 100
plot_every = 10
chunk_len = 200
hidden_size = 100
batch_size = 1
num_layers = 1
embedding_size = 70
lr = 0.002

In [38]:
all_characters= string.printable
n_characters = len(all_characters)
print(all_characters)
print('num_chars =', n_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

num_chars = 100


In [34]:
all_characters

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'

In [39]:
file = unidecode.unidecode(open('./data/input.txt').read())
file_len = len(file)
print('file_len =', file_len)

file_len = 1115394


In [40]:
def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

In [42]:
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return tensor

print(char_tensor('ABCdef'))

tensor([36, 37, 38, 13, 14, 15])


In [43]:
def random_training_set():
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

(tensor([78, 94, 11, 30, 29, 94, 34, 14, 29, 94, 22, 34, 94, 18, 23, 32, 10, 27,
         13, 94, 28, 24, 30, 21, 96, 51, 14, 27, 28, 30, 10, 13, 14, 28, 94, 22,
         14, 94, 18, 29, 94, 18, 28, 94, 24, 29, 17, 14, 27, 32, 18, 28, 14, 77,
         94, 17, 24, 32, 14, 68, 14, 27, 94, 18, 29, 94, 11, 14, 73, 96, 44, 94,
         12, 10, 23, 23, 24, 29, 94, 11, 30, 29, 94, 11, 14, 94, 28, 10, 13, 78,
         94, 28, 24, 94, 17, 14, 10, 31, 34, 94, 28, 10, 13, 96, 36, 28, 73, 94,
         29, 17, 24, 30, 16, 17, 94, 24, 23, 94, 29, 17, 18, 23, 20, 18, 23, 16,
         94, 24, 23, 94, 23, 24, 94, 29, 17, 24, 30, 16, 17, 29, 94, 44, 94, 29,
         17, 18, 23, 20, 73, 96, 48, 10, 20, 14, 28, 94, 22, 14, 94, 32, 18, 29,
         17, 94, 17, 14, 10, 31, 34, 94, 23, 24, 29, 17, 18, 23, 16, 94, 15, 10,
         18, 23, 29, 94, 10, 23, 13, 94, 28, 17, 27, 18, 23, 20, 75, 96, 96, 37,
         56, 54]),
 tensor([94, 11, 30, 29, 94, 34, 14, 29, 94, 22, 34, 94, 18, 23, 32, 10, 27, 13,
         

In [77]:
class RNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.embedding_size = embedding_size
        
        self.encoder = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.GRU(embedding_size, hidden_size, num_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        
    
    def forward(self, input, hidden):
        out = self.encoder(input.view(1, -1))
        out, hidden = self.rnn(out, hidden)
        out = self.decoder(out.view(batch_size, -1))
        return out, hidden
    
    
    def init_hidden(self):
        hidden = torch.zeros(self.num_layers, batch_size, hidden_size)
        return hidden

In [78]:
model = RNN(input_size=n_characters,
           embedding_size=embedding_size,
           hidden_size=hidden_size,
           output_size=n_characters,
           num_layers=2)

In [79]:
inp = char_tensor("A")
hidden = model.init_hidden()
out, hidden = model(inp, hidden)

In [80]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()

In [81]:
for i in tqdm(range(num_epochs)):
    total = char_tensor(random_chunk())
    inp = total[:-1]
    label = total[1:]
    hidden = model.init_hidden()
    
    loss = torch.tensor([0]).type(torch.FloatTensor)
    optimizer.zero_grad()
    for j in range(chunk_len-1):
        x = inp[j]
        y_ = label[j].unsqueeze(0).type(torch.LongTensor)
        y, hidden = model(x, hidden)
        y = y
        hidden = hidden
        loss += loss_func(y, y_)
        
    loss.backward()
    optimizer.step()

In [85]:
class RNN(nn.Module):
    def __init__(self, input_size, embedding_sie, hidden_size, output_size, num_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.embedding_size = embedding_size
        
        self.encoder = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        
    
    def forward(self, input, hidden, cell):
        out = self.encoder(input.view(batch_size, -1))
        out, (hidden, cell) = self.rnn(out, (hidden, cell))
        out = self.decoder(out.view(batch_size, -1))
        return out, hidden, cell
    
    
    def init_hidden(self):
        hidden = torch.zeros(num_layers, batch_size, hidden_size)
        cell = torch.zeros(num_layers, batch_size, hidden_size)
        return hidden, cell
    

model = RNN(n_characters, embedding_size, hidden_size, n_characters, num_layers)

In [86]:
def test():
    start_str = "b"
    inp = char_tensor(start_str)
    hidden,cell = model.init_hidden()
    x = inp

    print(start_str,end="")
    for i in range(200):
        output,hidden,cell = model(x,hidden,cell)

        output_dist = output.data.view(-1).div(0.8).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]

        print(predicted_char,end="")

        x = char_tensor(predicted_char)

In [88]:
for i in tqdm(range(num_epochs)):
    inp,label = random_training_set()
    hidden,cell = model.init_hidden()

    loss = torch.tensor([0]).type(torch.FloatTensor)
    optimizer.zero_grad()
    for j in range(chunk_len-1):
        x  = inp[j]
        y_ = label[j].unsqueeze(0).type(torch.LongTensor)
        y,hidden,cell = model(x,hidden,cell)
        loss += loss_func(y,y_)

    loss.backward()
    optimizer.step()
    
    if i % 100 == 0:
        print("\n",loss/chunk_len,"\n")
        test()
        print("\n\n")


 tensor([4.5758], grad_fn=<DivBackward0>) 

?MdFuC(+sWkLc"oY"(co,9|>f	SfRMA.0@D+h+y7QD4L*Kg$KQI[*!.m/{2S5%d
#\r	dg/6vON~mkS+fd~6;14{JpH
_28C@=&X;Iy-a^f<.y(ZLk.[M*"/+x*`[(`
~Jhr^B(ackYr5!NQ%V=8V'+zN}O



 tensor([4.5799], grad_fn=<DivBackward0>) 

bA"zAe,"OaXcvyhhV(#U*rD}4,n4Z@$RPu4l+>8dOh
z.*q]nez;73;wwm&j}j;uvk-HL~@{RD,Ea}4 Eu4Rq<zHYu~0~5d{z,*HR&8"6!4/OE0!h
u*S
B;2-5fQX/]/_jv-#
T$w` ?,tS>BE 



 tensor([4.5725], grad_fn=<DivBackward0>) 

&}wgLPGjlLyGut3D1vqK`m%oxB^:W>&Du+y"6;yR ]f1}d_*,|Ar1>-V"^)Xdl'8j$P1Y\Em<XCq[xOp Wa8LqeRc9<hb\~yJ|T~uW/w<a)O8._lZ:j>SaCO{s1ASMZ#Z"{d<dR2U~;l6



 tensor([4.5815], grad_fn=<DivBackward0>) 

bfp"rBggl^TmWU<1SMj{T<")O*Yc(}WZKW}e	;;,1pPNZRP_4&6 4{bTsN)Z"ek%5;PHuJ.D`h;~RlQxnDe+Aq#bqo V}J8GNSC*X4?;now=}-O"e
oV+sAW*`&V?wlcglG	x;Cz YkA&a>&q`s
;x



 tensor([4.5807], grad_fn=<DivBackward0>) 

b_V'>V$]GS
^JnxOy<lcHW2:w|)@	d_$+
2(m}0|A0iN3`4vy213W]C%#
qdw[W$Ad(Xcez5_=K,4dn	;wH%pr]L;#fHV!^1m6Va~^i+>9l>yx#S'ce#koX}IRX# 	R[JO'z;>d }Z;	\n)\sL5&"%y